## Задание 5.1

Набор данных тут: https://github.com/sismetanin/rureviews, также есть в папке [Data](https://drive.google.com/drive/folders/1YAMe7MiTxA-RSSd8Ex2p-L0Dspe6Gs4L). Те, кто предпочитает работать с английским языком, могут использовать набор данных `sms_spam`.

Применим полученные навыки и решим задачу анализа тональности отзывов. 

Нужно повторить весь пайплайн от сырых текстов до получения обученной модели.

Обязательные шаги предобработки:
1. токенизация
2. приведение к нижнему регистру
3. удаление стоп-слов
4. лемматизация
5. векторизация (с настройкой гиперпараметров)
6. построение модели
7. оценка качества модели

Обязательно использование векторайзеров:
1. мешок n-грамм (диапазон для n подбирайте самостоятельно, запрещено использовать только униграммы).
2. tf-idf ((диапазон для n подбирайте самостоятельно, также нужно подбирать параметры max_df, min_df, max_features)
3. символьные n-граммы (диапазон для n подбирайте самостоятельно)

В качестве классификатора нужно использовать наивный байесовский классификатор. 

Для сравнения векторайзеров между собой используйте precision, recall, f1-score и accuracy. Для этого сформируйте датафрейм, в котором в строках будут разные векторайзеры, а в столбцах разные метрики качества, а в  ячейках будут значения этих метрик для соответсвующих векторайзеров.

In [ ]:
import numpy as np
import pandas as pd

from sklearn.metrics import * 
from sklearn.model_selection import train_test_split 

In [ ]:
!wget https://www.dropbox.com/s/maw1n18jz7x8jq9/sms_spam.csv

--2021-12-25 20:29:42--  https://www.dropbox.com/s/maw1n18jz7x8jq9/sms_spam.csv
Resolving www.dropbox.com (www.dropbox.com)... 162.125.5.18, 2620:100:601d:18::a27d:512
Connecting to www.dropbox.com (www.dropbox.com)|162.125.5.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/maw1n18jz7x8jq9/sms_spam.csv [following]
--2021-12-25 20:29:43--  https://www.dropbox.com/s/raw/maw1n18jz7x8jq9/sms_spam.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucaba9ea14e61428b79f58200c6e.dl.dropboxusercontent.com/cd/0/inline/BchqBY9CdG1lbrB1rkCrp6GB5h0nfn2VCXisM9ERDgID07NFfkJ9kIubK3qc3L7TVXYI_hVmM7DGZggIfEJy3yredWa6gmC7Nd7NlcjyzjDFs4pGI-Q9zrHBP8Y2QEU2VASij5ygiONLZ1FpEkdQyVyY/file# [following]
--2021-12-25 20:29:43--  https://ucaba9ea14e61428b79f58200c6e.dl.dropboxusercontent.com/cd/0/inline/BchqBY9CdG1lbrB1rkCrp6GB5h0nfn2VCXisM9ERDgID07NFfkJ9kIubK3qc3L7TVXYI_hVmM7DGZggIfEJy3yredW

In [ ]:
!pip install pymorphy2

     |████████████████████████████████| 55 kB 2.2 MB/s 
     |████████████████████████████████| 8.2 MB 6.1 MB/s 


In [ ]:
from pymorphy2 import MorphAnalyzer
pymorphy2_analyzer = MorphAnalyzer()

In [ ]:
data = pd.read_csv("sms_spam.csv")
print(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5559 entries, 0 to 5558
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   type    5559 non-null   object
 1   text    5559 non-null   object
dtypes: object(2)
memory usage: 87.0+ KB
None


In [ ]:
import re
import nltk
from nltk.tokenize import word_tokenize 
from nltk.corpus import stopwords

In [ ]:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
noise = stopwords.words('english')

In [ ]:
print(data.iloc[0])

type                                                  ham
text    Hope you are having a good week. Just checking in
Name: 0, dtype: object


In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
for i in range(data.shape[0]):
  data.iloc[i][1] = data.iloc[i][1].lower()
  data.iloc[i][1] = data.iloc[i][1].split()
  for j in range(len(data.iloc[i][1])):
    pol = pymorphy2_analyzer.parse(data.iloc[i][1][j])
    data.iloc[i][1][j] = pol[0].normal_form
  data.iloc[i][1] = " ".join(data.iloc[i][1])

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(data.text, data.type, train_size = 0.7)
def test_vectorize(vectorizer):
  # x_train, x_test, y_train, y_test = train_test_split(data.text, data.type, train_size = 0.7)
  smart_vectorized_x_train = vectorizer.fit_transform(x_train)
  
  clf = MultinomialNB()
  clf.fit(smart_vectorized_x_train, y_train)

  smart_vectorized_x_test = vectorizer.transform(x_test)

  y_pred = clf.predict(smart_vectorized_x_test)
  return accuracy_score(y_test, y_pred), f1_score(y_test, y_pred, average='weighted', labels=np.unique(y_pred)), precision_score(y_test, y_pred, average='weighted', labels=np.unique(y_pred)), recall_score(y_test, y_pred, average='macro')


In [ ]:
output_table = pd.DataFrame(columns=['Vectorizer', 'accuracy', 'f1_score', 'precision', 'recall'])
count_table = pd.DataFrame(columns=['Vectorizer', 'accuracy', 'f1_score', 'precision', 'recall'])
ngrams = [(1,1), (2,2), (3,3), (4,4), (1,2), (1,3), (1,4), (2, 3), (2,4)]
for i in range(len(ngrams)):
  count_vectorizer = CountVectorizer(ngram_range=ngrams[i], stop_words=noise)
  acc, f1, prec, rec = test_vectorize(count_vectorizer)
  # print("CountVectorizer: ", "ngrams range: ", str(ngrams[i]), acc, f1, prec, rec)
  output_table.loc[output_table.shape[0]] = ["CountVectorizer: " + "Ngrams range: " + str(ngrams[i])] + [acc] + [f1] + [prec] + [rec]
  count_table.loc[output_table.shape[0]] = ["CountVectorizer: " + "Ngrams range: " + str(ngrams[i])] + [acc] + [f1] + [prec] + [rec]
print("Count Vectorizer results:")
pd.set_option('display.expand_frame_repr', False)
with pd.option_context('display.max_rows', None, 'display.max_columns', None): 
    print(count_table)


Count Vectorizer results:
                              Vectorizer  accuracy  f1_score  precision    recall
1  CountVectorizer: Ngrams range: (1, 1)  0.985012  0.984909   0.984882  0.960691
2  CountVectorizer: Ngrams range: (2, 2)  0.972422  0.971304   0.972568  0.903595
3  CountVectorizer: Ngrams range: (3, 3)  0.942446  0.935578   0.945476  0.784722
4  CountVectorizer: Ngrams range: (4, 4)  0.937650  0.929139   0.941831  0.764706
5  CountVectorizer: Ngrams range: (1, 2)  0.985012  0.984818   0.984889  0.954940
6  CountVectorizer: Ngrams range: (1, 3)  0.985612  0.985411   0.985518  0.955286
7  CountVectorizer: Ngrams range: (1, 4)  0.985612  0.985411   0.985518  0.955286
8  CountVectorizer: Ngrams range: (2, 3)  0.973621  0.972551   0.973877  0.906203
9  CountVectorizer: Ngrams range: (2, 4)  0.973621  0.972551   0.973877  0.906203


In [ ]:
max_features = [None]
df_boundaries = [(1, 1), (1, 100)]
tfid_table = pd.DataFrame(columns=['Vectorizer', 'Vectorizer_params', 'accuracy', 'f1_score', 'precision', 'recall'])
for i in range(len(ngrams)):
  for j in range(len(df_boundaries)):
    min_df = df_boundaries[j][0]
    max_df = df_boundaries[j][1]
    for f in range(len(max_features)):
      max_feature = max_features[f]
      count_vectorizer = TfidfVectorizer(ngram_range=ngrams[i], stop_words=noise, min_df=min_df, max_df=max_df, max_features=max_feature)
      acc, f1, prec, rec = test_vectorize(count_vectorizer)
      output_table.loc[output_table.shape[0]] = ["TfidfVectorizer: "  "Ngrams range: " + str(ngrams[i]) + ", min_df: " + str(min_df) + ", max_df: " + str(max_df)] + [acc] + [f1] + [prec] + [rec]
      tfid_table.loc[output_table.shape[0]] = ["TfidfVectorizer: " + "Ngrams range: " + str(ngrams[i])] + ["min_df: " + str(min_df) + ", max_df: " + str(max_df)] + [acc] + [f1] + [prec] + [rec]
print("Tfid vectorizer results:")
# pd.set_option('display.expand_frame_repr', False)
with pd.option_context('display.max_rows', None, 'display.max_columns', None): 
    print(tfid_table)

Tfid vectorizer results:
                               Vectorizer       Vectorizer_params  accuracy  f1_score  precision    recall
10  TfidfVectorizer: Ngrams range: (1, 1)    min_df: 1, max_df: 1  0.867506  0.929053   0.867506  0.500000
11  TfidfVectorizer: Ngrams range: (1, 1)  min_df: 1, max_df: 100  0.964029  0.961569   0.965461  0.864253
12  TfidfVectorizer: Ngrams range: (2, 2)    min_df: 1, max_df: 1  0.872302  0.817291   0.888687  0.518100
13  TfidfVectorizer: Ngrams range: (2, 2)  min_df: 1, max_df: 100  0.940048  0.932284   0.943923  0.773756
14  TfidfVectorizer: Ngrams range: (3, 3)    min_df: 1, max_df: 1  0.869305  0.810285   0.886417  0.506787
15  TfidfVectorizer: Ngrams range: (3, 3)  min_df: 1, max_df: 100  0.909472  0.888230   0.918027  0.658371
16  TfidfVectorizer: Ngrams range: (4, 4)    min_df: 1, max_df: 1  0.867506  0.929053   0.867506  0.500000
17  TfidfVectorizer: Ngrams range: (4, 4)  min_df: 1, max_df: 100  0.898082  0.869059   0.908797  0.615385
18  TfidfVec

In [ ]:
count_char = pd.DataFrame(columns=['Vectorizer', 'accuracy', 'f1_score', 'precision', 'recall'])
for i in range(len(ngrams)):
  count_vectorizer = CountVectorizer(ngram_range=ngrams[i], stop_words=noise, analyzer='char')
  acc, f1, prec, rec = test_vectorize(count_vectorizer)
  count_char.loc[output_table.shape[0]] = ["CountVectorizer(char): " + "Ngrams range: " + str(ngrams[i])] + [acc] + [f1] + [prec] + [rec]
  output_table.loc[output_table.shape[0]] = ["CountVectorizer(char): " + "Ngrams range: " + str(ngrams[i])] + [acc] + [f1] + [prec] + [rec]
print("Count vectorizer(char) results:")
pd.set_option('display.expand_frame_repr', False)
with pd.option_context('display.max_rows', None, 'display.max_columns', None): 
    print(count_char)


Count vectorizer(char) results:
                                     Vectorizer  accuracy  f1_score  precision    recall
27  CountVectorizer(char): Ngrams range: (1, 1)  0.970024  0.969480   0.969424  0.917548
28  CountVectorizer(char): Ngrams range: (2, 2)  0.983813  0.983570   0.983680  0.950415
29  CountVectorizer(char): Ngrams range: (3, 3)  0.983213  0.982944   0.983077  0.948153
30  CountVectorizer(char): Ngrams range: (4, 4)  0.983813  0.983604   0.983657  0.952332
31  CountVectorizer(char): Ngrams range: (1, 2)  0.981415  0.981175   0.981195  0.947116
32  CountVectorizer(char): Ngrams range: (1, 3)  0.983213  0.982979   0.983050  0.950070
33  CountVectorizer(char): Ngrams range: (1, 4)  0.983213  0.982944   0.983077  0.948153
34  CountVectorizer(char): Ngrams range: (2, 3)  0.983213  0.982944   0.983077  0.948153
35  CountVectorizer(char): Ngrams range: (2, 4)  0.985012  0.984818   0.984889  0.954940


In [ ]:
# print(output_table)

## Задание 5.2 Регулярные выражения

Регулярные выражения - способ поиска и анализа строк. Например, можно понять, какие даты в наборе строк представлены в формате DD/MM/YYYY, а какие - в других форматах. 

Или бывает, например, что перед работой с текстом, надо почистить его от своеобразного мусора: упоминаний пользователей, url и так далее.

Навык полезный, давайте в нём тоже потренируемся.

Для работы с регулярными выражениями есть библиотека **re**

In [ ]:
import re

В регулярных выражениях, кроме привычных символов-букв, есть специальные символы:
* **?а** - ноль или один символ **а**
* **+а** - один или более символов **а**
* **\*а** - ноль или более символов **а** (не путать с +)
* **.** - любое количество любого символа

Пример:
Выражению \*a?b. соответствуют последовательности a, ab, abc, aa, aac НО НЕ abb!

Рассмотрим подробно несколько наиболее полезных функций:

### findall
возвращает список всех найденных непересекающихся совпадений.

Регулярное выражение **ab+c.**: 
* **a** - просто символ **a**
* **b+** - один или более символов **b**
* **c** - просто символ **c**
* **.** - любой символ


In [ ]:
result = re.findall('ab+c.', 'abcdefghijkabcabcxabc') 
print(result)

['abcd', 'abca']


Вопрос на внимательность: почему нет abcx?

**Задание**: вернуть список первых двух букв каждого слова в строке, состоящей из нескольких слов.

In [ ]:
result = re.findall(r'\b\w{2}', 'Lorem ipsum dolor sit amet, consectetur adipiscing elit,\n sed do eiusmod tempor incididunt ut labore et\n dolore magna aliqua.\n')
print(result)

['Lo', 'ip', 'do', 'si', 'am', 'co', 'ad', 'el', 'se', 'do', 'ei', 'te', 'in', 'ut', 'la', 'et', 'do', 'ma', 'al']


### split
разделяет строку по заданному шаблону


In [ ]:
result = re.split(',', 'itsy, bitsy, teenie, weenie') 
print(result)

['itsy', ' bitsy', ' teenie', ' weenie']


можно указать максимальное количество разбиений

In [ ]:
result = re.split(',', 'itsy, bitsy, teenie, weenie', maxsplit=2) 
print(result)

['itsy', ' bitsy', ' teenie, weenie']


**Задание**: разбейте строку, состоящую из нескольких предложений, по точкам, но не более чем на 3 предложения.

In [ ]:
text = 'предложение бла бла бла. sentence 2. TOday is a good day. Вечер добрый. До завтра.'
result = re.split('\.', text, maxsplit=2)
print(result)

['предложение бла бла бла', ' sentence 2', ' TOday is a good day. Вечер добрый. До завтра.']


### sub
ищет шаблон в строке и заменяет все совпадения на указанную подстроку

параметры: (pattern, repl, string)

In [ ]:
result = re.sub('a', 'b', 'abcabc')
print (result)

bbcbbc


**Задание**: напишите регулярное выражение, которое позволит заменить все цифры в строке на "DIG".

In [ ]:
result = re.sub('[0-9]', 'DIG', 'asdknasodjn112ejndasjndo12ndo12jnd1o2jnd')
print(result)

asdknasodjnDIGDIGDIGejndasjndoDIGDIGndoDIGDIGjndDIGoDIGjnd


**Задание**: напишите  регулярное выражение, которое позволит убрать url из строки.

In [ ]:
result = re.sub('(http|https)://[^ "]+', '', 'https://docs.google.com/spreadsheets/d/1qo03lRyhl5xM4_Kx3jzJSb9K2vRGm9dXIqbUDmrkxMc/edit#gid=57874460 asdasdhttps://piazza.com/class/kt725hi4odv52p asdasldm1212')
print(result)

 asdasd asdasldm1212


### compile
компилирует регулярное выражение в отдельный объект

In [ ]:
# Пример: построение списка всех слов строки:
prog = re.compile('[А-Яа-яё\-]+')
prog.findall("Слова? Да, больше, ещё больше слов! Что-то ещё.")

['Слова', 'Да', 'больше', 'ещё', 'больше', 'слов', 'Что-то', 'ещё']

**Задание**: для выбранной строки постройте список слов, которые длиннее трех символов.

In [ ]:
prog = re.compile('[А-Яа-яё\-]{4,}')
prog.findall("Слова? Да, больше, ещё больше слов! Что-то ещё.")

['Слова', 'больше', 'больше', 'слов', 'Что-то']

**Задание**: вернуть список доменов (@gmail.com) из списка адресов электронной почты:

```
abc.test@gmail.com, xyz@test.in, test.first@analyticsvidhya.com, first.test@rest.biz
```

In [ ]:
text = 'abc.test@gmail.com, xyz@test.in, test.first@analyticsvidhya.com, first.test@rest.biz'
prog = re.compile('@[\w.]+')
prog.findall(text)

['@gmail.com', '@test.in', '@analyticsvidhya.com', '@rest.biz']